In [ ]:
import matplotlib.pyplot as plt
import matplotlib.style
import csv
import numpy as np
import datetime
from conf import *
import pickle
import matplotlib.dates as mdates
matplotlib.style.use('ggplot')

In [ ]:
times, values = pickle.load(open("tmp/plot_data.p", "rb"))

In [ ]:
fig, ax = plt.subplots(1, figsize=(7, 7))
fitstart = 0
end = 540

ax.grid(True)
ax.set_xlim(0, end)
ax.set_ylim(0, max(values[:end, 1]))
ax.fill_between(times[:end], 0, values[:end, 1] + values[:end, 0], color=np.array(agent_colors[0]) / 255)

endemic = 10 * np.sin(0.03*np.array(times)) + 50
endemic_line = ax.plot(times[:end], np.ones(len(times[:end]))*50, color=(0.2, 0.2,0.2))

pandemic = ax.fill_between(times[:end], 0, values[:end, 1], color=np.array(agent_colors[1]) / 255)

ax.legend([endemic_line[0], pandemic], ["Endemie", "Pandemie"], loc=2)


fit = np.polyfit(times[fitstart:end], np.log(values[fitstart:end, 1]), 1, w=np.log(values[fitstart:end, 1]))
exp = fit[1] + np.exp(fit[0]*np.array(times[:end]))

#ax.axvline(x=200)

#ax.scatter([200, 200], [7, 50], marker="o",zorder=10)

#ax.plot(times[:end], exp, color=(0.2, 0.2, 0.2))

fig.tight_layout()
fig.savefig("nachtrag/endemic0.png", dpi=300)

In [ ]:
with open('germany.csv', newline='') as f:
    reader = csv.reader(f)
    germany = np.array(list(reader)[0], dtype=np.int)
    
with open('italy.csv', newline='') as f:
    reader = csv.reader(f)
    italy = np.array(list(reader)[0], dtype=np.int)

In [ ]:
start_date = datetime.date(year=2020, month=1, day=22)

In [ ]:
dates = []
for i in range(len(germany)):
    dates.append(start_date + datetime.timedelta(days=i))

In [ ]:
fig, ax = plt.subplots(1, figsize=(7, 7))

start = 39

ax.grid(True)
ax.set_xlim(dates[start], dates[-1])
ax.set_ylim(0, max(germany)-1000)
ax.fill_between(dates, 0, max(germany), color=np.array(agent_colors[0]) / 255)
krank = ax.fill_between(dates, 0, germany, color=np.array(agent_colors[1]) / 255)

ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m'))

fig.autofmt_xdate()

date_idxs = np.linspace(0, len(dates)-start-1, len(dates)-start)
fit = np.polyfit(date_idxs, np.log(germany[start:]), 1)
exp = np.exp(fit[0]*np.array(date_idxs))*germany[start]
ax.plot(dates[start:], exp, color=(0.2, 0.2, 0.2))

ax.legend([krank], ["Bestätigte Fälle von SARS-CoV-2 in Deutschland"], loc=2)

fig.tight_layout()
fig.savefig("nachtrag/deutschland_fit.png", dpi=300)

In [ ]:
for italy_offset in range(0, 8):

    fig, ax = plt.subplots(1, figsize=(7, 7))

    start = 39

    ax.grid(True)
    ax.set_xlim(dates[start], dates[-1])
    ax.set_ylim(0, max(italy)-1000)

    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m'))

    fig.autofmt_xdate()

    #ax.fill_between(dates, 0, italy, color=np.array(agent_colors[1]) / 255)

    date_idxs = np.linspace(0, len(dates)-start-1, len(dates)-start)
    fit_germany = np.polyfit(date_idxs, np.log(germany[start:]), 1)
    exp_germany = np.exp(fit_germany[0]*np.array(date_idxs))*germany[start]
    line_germany = ax.plot(dates[start:], exp_germany, color=(0.2, 0.2, 0.2))

    fit_italy = np.polyfit(date_idxs, np.log(italy[start:]), 1)

    all_date_idxs = np.linspace(0, len(dates)-1, len(dates))
    exp_italy = np.exp(fit_italy[0]*np.array(all_date_idxs))*1.8

    italy_string = "Italien"
    if italy_offset>0:
        italy_string += " (um {} Tage verschoben)".format(italy_offset)
        line_italy = ax.plot(dates[italy_offset:], exp_italy[:-italy_offset], color=(0.1, 0.1, 1))
    else:
        line_italy = ax.plot(dates, exp_italy, color=(0.1, 0.1, 1))


    fig.legend(("Deutschland", italy_string), loc=2, bbox_to_anchor=(0.1, 0.46, 0.5, 0.5), facecolor="w")
    fig.tight_layout()
    
    fig.savefig("tmp/germany_italy_offset{}.png".format(italy_offset), dpi=300)

In [ ]:
print("Anstieg pro Tag: \n \t Deutschland: {:1.2f}% \n \t Italien: {:1.2f}%".format((np.exp(fit_germany[0])-1)*100, (np.exp(fit_italy[0])-1)*100))

In [ ]:
np.log(100)/fit_germany[0]

In [ ]:
np.exp(fit_germany[0]*7)